In [1]:
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [2]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x
ch4str="C 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [3]:
CH4=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf = scf.RHF(CH4)
mf.scf()
ge0=mf.Gradients().grad_elec()
g0=mf.Gradients().grad()

converged SCF energy = -40.2151804034982
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000    -0.0000000000
1 H     0.0000000000    -0.0000000000    -0.0000007926
2 H     0.0000000000    -0.0000007473     0.0000002642
3 H    -0.0000006472     0.0000003736     0.0000002642
4 H     0.0000006472     0.0000003736     0.0000002642
----------------------------------------------


In [4]:
CH4_T=FcM(fcs=[1,0,0,0,-1],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_T = scf.RHF(CH4_T)
mf_T.scf(dm0=mf_T.init_guess_by_1e())
geT=mf_T.Gradients().grad_elec()
gnT=mf_T.Gradients().grad_nuc()

converged SCF energy = -56.2053208898249


In [5]:
hT=mf_T.Hessian().hess()

In [6]:
ch4coords=CH4.atom_coords()

In [21]:
#ges=[]
#af=[]
#gn=[]
#hs=[]
dl=.1
#for i in range(-2,3):
#    CH4_p=FcM(fcs=[i*dl,0,0,0,-i*dl],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
#    mf_p = scf.RHF(CH4_p)
#    mf_p.scf(dm0=mf_p.init_guess_by_1e())
#    hs.append(mf_p.Hessian().hess())
#    ges.append(mf_p.Gradients().grad_elec())
#    af.append(aaff(mf_p,[1,0,0,0,-1]))
#    gn.append(alc_deriv_grad_nuc(CH4_p,[1,0,0,0,-1]))

In [22]:
#gns=np.asarray(gn)
#ges=np.asarray(ges)
#afs=np.asarray(af)
coefs5p=get_coeffs(5)

In [23]:
#np.save("ch4conv",(gns,ges,afs))
#np.save("ch4convhs",hs)
gns,ges,afs=np.load("ch4conv.npy")
hs=np.load("ch4convhs.npy")

In [24]:
gn_1=gns[2]+mf.Gradients().grad_nuc()
gn_2=gn_1+np.einsum("i,ijk->jk",coefs5p[1],gns)/dl/2
gn_3=gn_2+np.einsum("i,ijk->jk",coefs5p[2],gns)/dl**2/6
gn_4=gn_3+np.einsum("i,ijk->jk",coefs5p[3],gns)/dl**3/24
gn_5=gn_4+np.einsum("i,ijk->jk",coefs5p[4],gns)/dl**4/120

In [27]:
af_1=afs[2]+ge0
af_2=af_1+np.einsum("i,ijk->jk",coefs5p[1],afs)/dl/2
af_3=af_2+np.einsum("i,ijk->jk",coefs5p[2],afs)/dl**2/6
af_4=af_3+np.einsum("i,ijk->jk",coefs5p[3],afs)/dl**3/24
af_5=af_4+np.einsum("i,ijk->jk",coefs5p[4],afs)/dl**4/120

In [28]:
g_1=ge0
g_2=g_1+np.einsum("i,ijk->jk",coefs5p[1],ges)/dl
g_3=g_2+np.einsum("i,ijk->jk",coefs5p[2],ges)/dl**2/3
g_4=g_3+np.einsum("i,ijk->jk",coefs5p[3],ges)/dl**3/6
g_5=g_4+np.einsum("i,ijk->jk",coefs5p[4],ges)/dl**4/24

In [29]:
h_1=hs[2]
h_2=h_1+np.einsum("i,ijklm->jklm",coefs5p[1],hs)/dl
h_3=h_2+np.einsum("i,ijklm->jklm",coefs5p[2],hs)/dl**2/3
h_4=h_3+np.einsum("i,ijklm->jklm",coefs5p[3],hs)/dl**3/6
h_5=h_4+np.einsum("i,ijklm->jklm",coefs5p[4],hs)/dl**4/24

In [31]:
gn_1,gn_2,gn_3,gn_4,gn_5,gnT

(array([[ 1.17275091,  0.67708806,  0.47877356],
        [ 0.04488502,  0.02591438, -1.82230131],
        [ 0.04488502, -1.70944402,  0.63186608],
        [-1.45797944,  0.89359358,  0.63186608],
        [ 0.19545849,  0.11284801,  0.07979559]]),
 array([[ 1.36820940e+00,  7.89936064e-01,  5.58569148e-01],
        [ 4.48850208e-02,  2.59143788e-02, -1.82230131e+00],
        [ 4.48850208e-02, -1.70944402e+00,  6.31866080e-01],
        [-1.45797944e+00,  8.93593580e-01,  6.31866080e-01],
        [ 6.38378239e-16,  5.68989300e-16,  5.13478149e-16]]),
 array([[ 1.36820940e+00,  7.89936064e-01,  5.58569148e-01],
        [ 4.48850208e-02,  2.59143788e-02, -1.82230131e+00],
        [ 4.48850208e-02, -1.70944402e+00,  6.31866080e-01],
        [-1.45797944e+00,  8.93593580e-01,  6.31866080e-01],
        [ 8.27116153e-15, -1.24900090e-16, -1.80411242e-16]]),
 array([[ 1.36820940e+00,  7.89936064e-01,  5.58569148e-01],
        [ 4.48850208e-02,  2.59143788e-02, -1.82230131e+00],
        [ 4.48850

In [32]:
alc_differential_grad_nuc(CH4,[1,0,0,0,-1])+mf.Gradients().grad_nuc()

array([[ 1.3682094 ,  0.78993606,  0.55856915],
       [ 0.04488502,  0.02591438, -1.82230131],
       [ 0.04488502, -1.70944402,  0.63186608],
       [-1.45797944,  0.89359358,  0.63186608],
       [ 0.        ,  0.        ,  0.        ]])

In [33]:
1/norm(np.array([-0.07818339, -0.0451392 , -0.03191824]))

10.44335085333552

In [34]:
geT

array([[-1.40613462e+00, -8.11832199e-01, -5.74052053e-01],
       [-5.00934672e-02, -2.89214768e-02,  1.88573378e+00],
       [-5.00934672e-02,  1.76824637e+00, -6.55845358e-01],
       [ 1.50629954e+00, -9.27505400e-01, -6.55845358e-01],
       [ 2.20071916e-05,  1.27058580e-05,  8.98439833e-06]])

In [65]:
af_2,af_3,af_4,af_5

(array([[-1.48774252, -0.85894854, -0.60736834],
        [-0.05036408, -0.02907771,  1.88806535],
        [-0.05036408,  1.77039251, -0.65676985],
        [ 1.50802285, -0.92881283, -0.65676985],
        [ 0.08044783,  0.04644658,  0.03284269]]),
 array([[-1.42869742, -0.82485884, -0.58326328],
        [-0.04887797, -0.02821971,  1.8862004 ],
        [-0.04887797,  1.76892022, -0.65533926],
        [ 1.50749086, -0.92678968, -0.65533926],
        [ 0.01896251,  0.01094801,  0.00774141]]),
 array([[-1.3721997 , -0.79223987, -0.56019818],
        [-0.05028035, -0.02902937,  1.88610773],
        [-0.05028035,  1.76856296, -0.65607173],
        [ 1.50648028, -0.92782554, -0.65607173],
        [-0.03371989, -0.01946819, -0.01376609]]),
 array([[-1.40740884e+00, -8.12567873e-01, -5.74572254e-01],
        [-5.05010283e-02, -2.91567824e-02,  1.88540374e+00],
        [-5.05010284e-02,  1.76785676e+00, -6.55957190e-01],
        [ 1.50575835e+00, -9.27663554e-01, -6.55957190e-01],
        [ 2.652

In [118]:
(ges[3]-ges[1])/2/dl,afs[2]
np.einsum("ijk,i->jk",ges,get_coeffs(5)[1])/dl,afs[2]
ges[2]+np.einsum("ijk,i->jk",ges,get_coeffs(5)[1])/dl+np.einsum("ijk,i->jk",ges,get_coeffs(5)[2])/dl**2/2

array([[-1.48773676, -0.85894522, -0.60736599],
       [-0.05036315, -0.02907718,  1.88807684],
       [-0.05036315,  1.77040352, -0.65677317],
       [ 1.50803285, -0.92881753, -0.65677317],
       [ 0.08043021,  0.0464364 ,  0.03283549]])

In [121]:
af_2m

array([[-1.40729469, -0.81250197, -0.57452565],
       [-0.05036408, -0.02907771,  1.88806535],
       [-0.05036408,  1.77039251, -0.65676985],
       [ 1.50802285, -0.92881283, -0.65676985],
       [ 0.        ,  0.        ,  0.        ]])

In [119]:
geT

array([[-1.40613462e+00, -8.11832199e-01, -5.74052053e-01],
       [-5.00934672e-02, -2.89214768e-02,  1.88573378e+00],
       [-5.00934672e-02,  1.76824637e+00, -6.55845358e-01],
       [ 1.50629954e+00, -9.27505400e-01, -6.55845358e-01],
       [ 2.20071916e-05,  1.27058580e-05,  8.98439833e-06]])

In [110]:
ge0

array([[ 3.23968998e-14, -2.26721315e-14, -6.39709975e-14],
       [ 2.58027367e-14, -3.23376014e-14,  1.65621067e+00],
       [ 2.29809797e-14,  1.56149039e+00, -5.52070223e-01],
       [ 1.35229035e+00, -7.80745197e-01, -5.52070223e-01],
       [-1.35229035e+00, -7.80745197e-01, -5.52070223e-01]])

In [35]:
for x in af_1:
    print (norm(x))
print ("*****************")
for x in af_2:
    print (norm(x))
print ("*****************")
for x in af_3:
    print (norm(x))
print ("*****************")
for x in af_4: 
    print (norm(x))
print ("*****************")
for x in af_5:
    print (norm(x))
print ("*****************")
for x in geT:
    print (norm(x))

1.46744962160717
1.8913808091205273
1.8913808091203812
1.8913808091202287
0.2353401273908252
*****************
1.8221050195285113
1.8889607735837508
1.8889607735835168
1.8889607735828144
0.09852806464201305
*****************
1.7497898330266315
1.8870446047304834
1.887044604730739
1.8870446047294125
0.023224232810226866
*****************
1.6805945455608449
1.8870011067117587
1.8870011067129233
1.8870011067198456
0.04129826266343128
*****************
1.7237167608320894
1.88630530902746
1.8863053090216768
1.8863053090368966
0.003248692762176061
*****************
1.7221561587102268
1.8866207125064238
1.8866207125064067
1.8866207125064147
2.6953194985988992e-05


In [36]:
af_1m=af_1.copy()
af_2m=af_2.copy()
af_3m=af_3.copy()
af_4m=af_4.copy()
af_5m=af_5.copy()
af_1m[0]+=af_1[4]
af_1m[4]-=af_1[4]
af_2m[0]+=af_2[4]
af_2m[4]-=af_2[4]
af_3m[0]+=af_3[4]
af_3m[4]-=af_3[4]
af_4m[0]+=af_4[4]
af_4m[4]-=af_4[4]
af_5m[0]+=af_5[4]
af_5m[4]-=af_5[4]

In [37]:
for x in [af_1,af_2,af_3,af_4,af_5]:
    print (norm(x-geT))
for x in [af_1m,af_2m,af_3m,af_4m,af_5m]:
    print (norm(x-geT))

0.34719780596387567
0.14038819463728863
0.03617038303261577
0.0586148474930476
0.0037156836631159597
0.025461585795608834
0.004315187897880489
0.005099683502381754
0.0007932674597748324
0.0019599991702602864


In [42]:
hT[4,:,:,:]

array([[[-8.57459417e-05,  1.02379801e-04,  7.23934515e-05],
        [ 1.02379802e-04, -2.03963952e-04,  4.17963787e-05],
        [ 7.23934521e-05,  4.17963787e-05, -2.33518455e-04]],

       [[ 2.77508670e-05, -2.13966628e-05,  1.00063928e-05],
        [-2.13966628e-05,  5.24576051e-05,  5.77719359e-06],
        [-7.41259786e-05, -4.27966537e-05, -1.46915760e-05]],

       [[ 2.77508670e-05,  2.30189669e-06, -2.35084315e-05],
        [-7.70188637e-05, -1.88646497e-05, -1.75804287e-05],
        [ 4.53569234e-06,  3.09934186e-05,  5.66306789e-05]],

       [[-3.95641663e-05, -3.81545111e-05, -2.69793135e-05],
        [ 4.11662493e-05,  4.84503836e-05, -1.15686845e-05],
        [ 2.91089340e-05, -1.15686845e-05,  5.66306789e-05]],

       [[ 6.98083740e-05, -4.51305233e-05, -3.19120990e-05],
        [-4.51305233e-05,  1.21920613e-04, -1.84244590e-05],
        [-3.19120990e-05, -1.84244590e-05,  1.34948673e-04]]])

In [30]:
for x in [h_1,h_2,h_3,h_4,h_5]:
    print (norm(x-hT))
    for i in range(5):
        print (norm(x[i,0]-hT[i,0]),norm(x[i,1]-hT[i,1]),norm(x[i,2]-hT[i,2]),norm(x[i,3]-hT[i,3]),norm(x[i,4]-hT[i,4]))

0.7861385921413027
0.4120751248570388 0.079067050432679 0.07906705049226175 0.07906705048439593 0.3478333782371953
0.079067050432679 0.07903407614041444 0.007036455895383733 0.00703645589539195 0.03737669734729944
0.07906705049226179 0.007036455895383734 0.07903407614041161 0.0070364558953936755 0.03737669734729486
0.07906705048439587 0.007036455895391951 0.0070364558953936755 0.0790340761402393 0.0373766973473148
0.3478333782371953 0.03737669734729943 0.03737669734729486 0.0373766973473148 0.3759041603988789
0.38692501950099967
0.24114530991482078 0.025927957994344074 0.025927957998077827 0.025927957998676976 0.17102559237613876
0.025927957994344074 0.021026685447602025 0.004629241150961882 0.0046292411508779404 0.012738856095590204
0.025927957998077896 0.004629241150961882 0.021026685449138095 0.0046292411508405285 0.012738856095851687
0.02592795799867735 0.00462924115087794 0.0046292411508405285 0.02102668545030924 0.01273885609578579
0.17102559237614173 0.012738856095590226 0.01273

In [43]:
for x in [h_1,h_2,h_3,h_4,h_5]:
    print (norm(x-hT))

0.7861385921413027
0.38692501950099967
0.13966097930260138
0.06234891285082125
0.31076396530846406


In [45]:
h_1m=h_1.copy()
h_2m=h_2.copy()
h_3m=h_3.copy()
h_4m=h_4.copy()
h_5m=h_5.copy()
h_1m[4,:,:,:]=0
h_1m[:,4,:,:]=0
h_2m[4,:,:,:]=0
h_2m[:,4,:,:]=0
h_3m[4,:,:,:]=0
h_3m[:,4,:,:]=0
h_4m[4,:,:,:]=0
h_4m[:,4,:,:]=0
h_5m[4,:,:,:]=0
h_5m[:,4,:,:]=0

In [46]:
for x in [h_1m,h_2m,h_3m,h_4m,h_5m]:
    print (norm(x-hT))

0.4757647732364819
0.252269602880318
0.09114075321227076
0.04343783034932294
0.18135258961701115


In [57]:
h_4[4,0,:,:],h_5[0,0,:,:]-hT[0,0,:,:]

(array([[-1.32212702e-02,  9.79572965e-07,  6.92657704e-07],
        [ 9.79582959e-07, -1.32224014e-02,  3.99913625e-07],
        [ 6.92669389e-07,  3.99915150e-07, -1.32226841e-02]]),
 array([[0.12364832, 0.03911409, 0.02765784],
        [0.03911409, 0.07848326, 0.01596826],
        [0.02765784, 0.01596826, 0.06719199]]))